In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import plot
from tqdm import tqdm
from tabulate import tabulate
import pickle as pkl
pd.set_option("display.max_columns", None)

In [3]:
class Data:
    
    def __init__(self, path):
        self.df = {
            'raw': pd.read_pickle(path)
        }
        if 'time' in self.df['raw'].columns:
            self.df['raw']['time'] = [ x.replace(tzinfo=None) for x in self.df['raw']['time']]

    def __repr__(self) -> str:
        repr = str()
        for name, df in self.df.items():
            repr = repr + name + ':\n' + str(df.head(3)) + '\n'
        return repr

    def shorten(self, name: str, rows: int, direction: int, source: str='raw', cols: list=None):
        '''Create new dataframe with specified list of columns and number of rows
        direction: 1 if data should be selected from top and -1 if from bottom
        '''
        assert (direction != 1 or direction != -1), 'direction must be 1 (top) or -1 (bottom)'
        
        if cols == None:
            cols = self.df[source].columns
        if direction == 1:
            self.df[name] = self.df[source][cols].iloc[:rows].copy()
        else:
            self.df[name] = self.df[source][cols].iloc[-rows:].copy()
        self.df[name].reset_index(drop=True, inplace=True)

    def add_columns(self, name: str, cols: list):
        '''Add new columns to component dataframes
        '''        
        exist_cols = list(self.df[name].columns)
        cols = exist_cols + cols
        self.df[name] = self.df[name].reindex(columns = cols) 

    def prepare_fast_data(self, name: str):
        '''Prepare data as an array for fast processing
        fcols = {col1: col1_index, col2: col2_index, .... }     
        fdata = [array[col1], array[col2], array[col3], .... ]
        Accessed by: self.fdata[fcols[column_name]] for whole column or
                     self.fdata[fcols[column_name]][row_index] for a specific row item
        '''
        self.fcols = dict()
        for i in range(len(self.df[name].columns)):
            self.fcols[self.df[name].columns[i]] = i
        self.fastdf = [self.df[name][col].array for col in self.df[name].columns]

    def fdata(self, column: str, index: int=-1):
        assert index >= -1, 'Row index cannot be negative'
        if index == -1:
            return self.fastdf[self.fcols[column]]
        else:
            return self.fastdf[self.fcols[column]][index]
        
    def update_fdata(self, column: str, value, index: int=-1):
        assert index >= -1, 'Row index cannot be negative'
        if index == -1:
            self.fastdf[self.fcols[column]] = value
        else:
            self.fastdf[self.fcols[column]][index] = value


In [4]:
PATH = 'D:/Trading/forex_bot/outputs/'

In [5]:
files = ['inputs1.pkl', 'inputs2.pkl', 'inputs3.pkl']

In [6]:
dfs = list()
for f in files:
    dfs.append(pd.read_pickle(PATH + f))

In [7]:
inputs = pd.concat(dfs)
inputs.drop_duplicates(inplace=True)
inputs.reset_index(drop=True,inplace=True)
inputs

,sim_name,init_signal,cushion,risk,rr,margin_closeout,streak_limit
0,sim_1,1,1.5,0.001,1.5,True,1
1,sim_2,1,1.5,0.001,1.5,True,2
2,sim_3,-1,1.5,0.001,1.5,True,3
3,sim_4,-1,1.5,0.002,1.5,True,1
4,sim_5,-1,1.5,0.002,1.5,True,2
...,...,...,...,...,...,...,...
535,sim_2176,-1,4.0,0.003,3.0,True,8
536,sim_2177,1,4.0,0.003,3.0,True,9
537,sim_2178,-1,4.0,0.004,3.0,True,7
538,sim_2179,1,4.0,0.004,3.0,True,8


In [8]:
inputs.to_pickle(PATH + 'inputs.all.pkl')

In [9]:
d = Data(PATH + "inputs.all.pkl")

In [10]:
subset = ['init_signal', 'cushion', 'risk', 'rr', 'margin_closeout', 'streak_limit']
d.df['raw'].drop_duplicates(subset=subset, keep='first', inplace=True)
d.df['raw'].reset_index(drop=True, inplace=True)

In [11]:
cols = ['return_%', 'final_ac_bal', 'total_trades', 'avg_trade', 'avg_trade_duration', 'win_%', 'loss_%', 
        'expectancy', 'sharpe_ratio', 'sortino_ratio', 'calmar_ratio',
        'max_drawdown', 'max_ac_bal', 'min_ac_bal', 'max_margin_used', 'total_streaks', 'avg_trades_per_streak',
        'total_wins', 'max_win', 'avg_win', 'avg_win_duration', 'total_losses', 'max_loss', 'avg_loss', 'avg_loss_duration',
        'total_long', 'avg_long', 'avg_long_duration',
        'total_long_wins', 'avg_long_win', 'avg_long_win_duration',
        'total_long_losses', 'avg_long_loss', 'avg_long_loss_duration',
        'total_short', 'avg_short', 'avg_short_duration',
        'total_short_wins', 'avg_short_win', 'avg_short_win_duration',
        'total_short_losses', 'avg_short_loss', 'avg_short_loss_duration',
        
        ]
d.add_columns('raw', cols)
d.df['raw'].head(3)

,sim_name,init_signal,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
0,sim_1,1,1.5,0.001,1.5,True,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sim_2,1,1.5,0.001,1.5,True,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sim_3,-1,1.5,0.001,1.5,True,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
d.prepare_fast_data('raw')

In [13]:
d.fdata('sim_name', 0)

'sim_1'

In [14]:
def convert2date(date_time):
    return date_time.date().strftime("%Y-%m-%d")

In [15]:
def sharpe_ratio(returns: pd.Series, risk_free_rate: float=0) -> float:
    return (returns.mean() - risk_free_rate) / returns.std()

def sortino_ratio(returns: pd.Series, risk_free_rate: float=0, target_return: float=0) -> float:
    downside_returns = np.array([ret - risk_free_rate for ret in returns if ret < target_return])
    std_dev_downside = np.std(downside_returns, ddof=1) if len(downside_returns) > 0 else 0
    return (np.mean(returns) - risk_free_rate) / std_dev_downside if std_dev_downside != 0 else np.nan

def max_drawdown(returns: pd.Series) -> float:
    '''
    cum_returns: Calculates the cumulative returns using np.cumprod(1 + returns).
    peaks: Finds the peaks in the cumulative returns using np.maximum.accumulate(cum_returns).
    drawdowns: Calculates the drawdowns for each point by subtracting the peaks from the cumulative returns and dividing by the peaks.
    max_drawdown: Identifies the maximum drawdown by finding the minimum drawdown value using np.min(drawdowns).'''
    cum_returns = np.cumprod(1 + returns.values)
    peaks = np.maximum.accumulate(cum_returns)
    drawdowns = (cum_returns - peaks) / peaks
    max_drawdown = np.min(drawdowns)
    return max_drawdown

def calmar_ratio(returns: pd.Series, risk_free_rate: float=0):
    max_drawdown_value = max_drawdown(returns)
    annual_return = np.mean(returns) * 311  # Assuming 311 trading days per year for forex. Its usually 252 for stocks
    return annual_return / abs(max_drawdown_value)

In [16]:
def populate_results(d: Data, i: int, df: pd.DataFrame):
    d.update_fdata('return_%', (df['ac_bal'].iloc[-1] - df['ac_bal'].iloc[0]) / df['ac_bal'].iloc[0], i)
    d.update_fdata('final_ac_bal', df['ac_bal'].iloc[-1], i)
    d.update_fdata('max_ac_bal', df['ac_bal'].max(), i)
    d.update_fdata('min_ac_bal', df['ac_bal'].min(), i)
    d.update_fdata('max_margin_used', df['margin_used'].max(), i)
    d.update_fdata('total_streaks', df['streak_no'].iloc[-1], i)
    d.update_fdata('avg_trades_per_streak', df.groupby('streak_no')['trade_no'].max().mean(), i)
    
    trades = df[df.signal != 0].copy()
    trades['return'] = trades['ac_bal'].shift(-1) - trades['ac_bal']
    trades['return_%'] = trades['ac_bal'].pct_change().shift(-1)
    trades['index'] = trades.index
    trades['duration'] = trades['index'].shift(-1) - trades['index']
    trades.dropna(inplace=True)
    # print(trades[['ac_bal', 'return', 'return_%']].head())

    d.update_fdata('sharpe_ratio', sharpe_ratio(trades['return_%']), i)
    d.update_fdata('sortino_ratio', sortino_ratio(trades['return_%']), i)
    d.update_fdata('calmar_ratio', calmar_ratio(trades['return_%']), i)
    d.update_fdata('max_drawdown', max_drawdown(trades['return_%']), i)

    d.update_fdata('total_trades', trades.shape[0], i)
    d.update_fdata('avg_trade', trades['return'].mean(), i)
    d.update_fdata('avg_trade_duration', trades['duration'].mean(), i)
    
    wins = trades[trades['return'] > 0]
    d.update_fdata('total_wins', wins.shape[0], i)
    d.update_fdata('max_win', wins['return'].max(), i)
    d.update_fdata('avg_win', wins['return'].mean(), i)
    d.update_fdata('avg_win_duration', wins['duration'].mean(), i)
    d.update_fdata('win_%', d.fdata('total_wins', i) / d.fdata('total_trades', i), i)

    losses = trades[trades['return'] < 0]
    d.update_fdata('total_losses', losses.shape[0], i)
    d.update_fdata('max_loss', losses['return'].min(), i)
    d.update_fdata('avg_loss', losses['return'].mean(), i)
    d.update_fdata('avg_loss_duration', losses['duration'].mean(), i)
    d.update_fdata('loss_%', d.fdata('total_losses', i) / d.fdata('total_trades', i), i)
    
    longs = trades[trades.signal == 1]
    d.update_fdata('total_long', longs.shape[0], i)
    d.update_fdata('avg_long', longs['return'].mean(), i)
    d.update_fdata('avg_long_duration', longs['duration'].mean(), i)

    long_wins = longs[longs['return'] > 0]
    d.update_fdata('total_long_wins', long_wins.shape[0], i)
    d.update_fdata('avg_long_win', long_wins['return'].mean(), i)
    d.update_fdata('avg_long_win_duration', long_wins['duration'].mean(), i)

    long_losses = longs[longs['return'] < 0]
    d.update_fdata('total_long_losses', long_losses.shape[0], i)
    d.update_fdata('avg_long_loss', long_losses['return'].mean(), i)
    d.update_fdata('avg_long_loss_duration', long_losses['duration'].mean(), i)

    shorts = trades[trades.signal == -1]
    d.update_fdata('total_short', shorts.shape[0], i)
    d.update_fdata('avg_short', shorts['return'].mean(), i)
    d.update_fdata('avg_short_duration', shorts['duration'].mean(), i)

    short_wins = shorts[shorts['return'] > 0]
    d.update_fdata('total_short_wins', short_wins.shape[0], i)
    d.update_fdata('avg_short_win', short_wins['return'].mean(), i)
    d.update_fdata('avg_short_win_duration', short_wins['duration'].mean(), i)

    short_losses = shorts[shorts['return'] < 0]
    d.update_fdata('total_short_losses', short_losses.shape[0], i)
    d.update_fdata('avg_short_loss', short_losses['return'].mean(), i)
    d.update_fdata('avg_short_loss_duration', short_losses['duration'].mean(), i)


    d.update_fdata('expectancy', d.fdata('win_%', i) * d.fdata('avg_win', i) + d.fdata('loss_%', i) * d.fdata('avg_loss', i), i)

In [17]:
sims = d.df['raw'].shape[0]
for i in tqdm(range(sims), desc=" Analysing... "):
# for i in tqdm(range(2), desc=" Analysing... "):
    data = pd.read_pickle(PATH + d.fdata('sim_name', i) + '.pkl')
    assert (d.fdata('sim_name', i) == data['sim_name'] and
            d.fdata('init_signal', i) == data['init_signal'] and
            d.fdata('cushion', i) == data['cushion'] and
            d.fdata('risk', i) == data['risk'] and
            d.fdata('rr', i) == data['rr'] and
            d.fdata('margin_closeout', i) == data['margin_closeout'] and
            d.fdata('streak_limit', i) == data['streak_limit']), f"Parameters mismatch for {d.fdata('sim_name', i)}"
    
    populate_results(d, i, data['results'])

 Analysing... :   0%|          | 0/540 [00:00<?, ?it/s]

 Analysing... : 100%|██████████| 540/540 [01:21<00:00,  6.62it/s]


In [18]:
with open(PATH + 'hedging_sim_analysis.pkl', 'wb') as file:
    pkl.dump(d.df['raw'], file)

In [19]:
d.df['raw'].head()

,sim_name,init_signal,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
0,sim_1,1,1.5,0.001,1.5,True,1,-10.23073,-9230.73,35475.0,-0.288393,16.752417,0.333531,0.666469,-0.288393,-0.002348,-0.002175,-0.013405,-10.086325,1016.16,-9233.16,33.333333,35476.0,1.000000,11832.0,16.93,1.876650,22.190754,23643.0,-14.00,-1.371875,14.030834,17695.0,-0.288110,16.812150,5873.0,1.888209,22.841989,11822.0,-1.369274,13.816613,17780.0,-0.288674,16.692970,5959.0,1.865258,21.548918,11821.0,-1.374477,14.245072
1,sim_2,1,1.5,0.001,1.5,True,2,-10.23073,-9230.73,35475.0,-0.288393,16.752417,0.333531,0.666469,-0.288393,-0.002348,-0.002175,-0.013405,-10.086325,1016.16,-9233.16,33.333333,21183.0,1.674739,11832.0,16.93,1.876650,22.190754,23643.0,-14.00,-1.371875,14.030834,17695.0,-0.288110,16.812150,5873.0,1.888209,22.841989,11822.0,-1.369274,13.816613,17780.0,-0.288674,16.692970,5959.0,1.865258,21.548918,11821.0,-1.374477,14.245072
2,sim_3,-1,1.5,0.001,1.5,True,3,-12.31986,-11319.86,35474.0,-0.347293,16.752889,0.333540,0.666460,-0.347293,0.006884,0.009342,0.007457,-12.065229,1023.16,-11321.50,66.666667,16746.0,2.118416,11832.0,22.58,2.279707,22.189571,23642.0,-24.54,-1.662015,14.032019,17694.0,-0.347382,16.812592,5873.0,2.288667,22.839775,11821.0,-1.657043,13.818120,17780.0,-0.347204,16.693476,5959.0,2.270876,21.548750,11821.0,-1.666986,14.245918
3,sim_4,-1,1.5,0.002,1.5,True,1,-3.51741,-2517.41,9466.0,-0.371584,62.779527,0.350518,0.649482,-0.371584,-0.009140,-0.006092,-0.472808,-3.486658,1014.47,-2522.64,33.333333,9467.0,1.000000,3318.0,17.89,3.463162,80.180229,6148.0,-11.95,-2.441148,53.388582,4714.0,-0.369650,61.044972,1640.0,3.512341,78.903659,3074.0,-2.440719,51.517241,4752.0,-0.373502,64.500210,1678.0,3.415095,81.427890,3074.0,-2.441578,55.259922
4,sim_5,-1,1.5,0.002,1.5,True,2,-3.51741,-2517.41,9466.0,-0.371584,62.779527,0.350518,0.649482,-0.371584,-0.009140,-0.006092,-0.472808,-3.486658,1014.47,-2522.64,33.333333,5734.0,1.651029,3318.0,17.89,3.463162,80.180229,6148.0,-11.95,-2.441148,53.388582,4714.0,-0.369650,61.044972,1640.0,3.512341,78.903659,3074.0,-2.440719,51.517241,4752.0,-0.373502,64.500210,1678.0,3.415095,81.427890,3074.0,-2.441578,55.259922


In [20]:
with open(PATH + 'hedging_sim_analysis.pkl', 'rb') as f:
    df_all = pkl.load(f)

In [21]:
df_all.drop_duplicates(subset=['init_signal', 'cushion', 'risk', 'rr', 'margin_closeout', 'streak_limit'],keep='first')
df_all

,sim_name,init_signal,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
0,sim_1,1,1.5,0.001,1.5,True,1,-10.230730,-9230.73000,35475.0,-0.288393,16.752417,0.333531,0.666469,-0.288393,-0.002348,-0.002175,-0.013405,-10.086325,1016.16000,-9233.16000,33.333333,35476.0,1.000000,11832.0,16.93000,1.876650,22.190754,23643.0,-14.00000,-1.371875,14.030834,17695.0,-0.288110,16.812150,5873.0,1.888209,22.841989,11822.0,-1.369274,13.816613,17780.0,-0.288674,16.692970,5959.0,1.865258,21.548918,11821.0,-1.374477,14.245072
1,sim_2,1,1.5,0.001,1.5,True,2,-10.230730,-9230.73000,35475.0,-0.288393,16.752417,0.333531,0.666469,-0.288393,-0.002348,-0.002175,-0.013405,-10.086325,1016.16000,-9233.16000,33.333333,21183.0,1.674739,11832.0,16.93000,1.876650,22.190754,23643.0,-14.00000,-1.371875,14.030834,17695.0,-0.288110,16.812150,5873.0,1.888209,22.841989,11822.0,-1.369274,13.816613,17780.0,-0.288674,16.692970,5959.0,1.865258,21.548918,11821.0,-1.374477,14.245072
2,sim_3,-1,1.5,0.001,1.5,True,3,-12.319860,-11319.86000,35474.0,-0.347293,16.752889,0.333540,0.666460,-0.347293,0.006884,0.009342,0.007457,-12.065229,1023.16000,-11321.50000,66.666667,16746.0,2.118416,11832.0,22.58000,2.279707,22.189571,23642.0,-24.54000,-1.662015,14.032019,17694.0,-0.347382,16.812592,5873.0,2.288667,22.839775,11821.0,-1.657043,13.818120,17780.0,-0.347204,16.693476,5959.0,2.270876,21.548750,11821.0,-1.666986,14.245918
3,sim_4,-1,1.5,0.002,1.5,True,1,-3.517410,-2517.41000,9466.0,-0.371584,62.779527,0.350518,0.649482,-0.371584,-0.009140,-0.006092,-0.472808,-3.486658,1014.47000,-2522.64000,33.333333,9467.0,1.000000,3318.0,17.89000,3.463162,80.180229,6148.0,-11.95000,-2.441148,53.388582,4714.0,-0.369650,61.044972,1640.0,3.512341,78.903659,3074.0,-2.440719,51.517241,4752.0,-0.373502,64.500210,1678.0,3.415095,81.427890,3074.0,-2.441578,55.259922
4,sim_5,-1,1.5,0.002,1.5,True,2,-3.517410,-2517.41000,9466.0,-0.371584,62.779527,0.350518,0.649482,-0.371584,-0.009140,-0.006092,-0.472808,-3.486658,1014.47000,-2522.64000,33.333333,5734.0,1.651029,3318.0,17.89000,3.463162,80.180229,6148.0,-11.95000,-2.441148,53.388582,4714.0,-0.369650,61.044972,1640.0,3.512341,78.903659,3074.0,-2.440719,51.517241,4752.0,-0.373502,64.500210,1678.0,3.415095,81.427890,3074.0,-2.441578,55.259922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,sim_2176,-1,4.0,0.003,3.0,True,8,1.318534,2318.53445,2273.0,0.580086,261.448306,0.229212,0.770788,0.580086,-0.012775,-0.010266,-3.588886,-2.463724,6931.59064,-3525.18327,7175.066667,583.0,3.900515,521.0,2055.65660,211.613791,441.865643,1752.0,-1095.63268,-62.175942,207.796804,1138.0,-0.258045,248.965729,262.0,206.664346,421.377863,876.0,-62.145792,197.399543,1135.0,1.420431,273.963877,259.0,216.620565,462.590734,876.0,-62.206093,218.194064
536,sim_2177,1,4.0,0.003,3.0,True,9,-12.102999,-11102.99902,2268.0,-5.336419,262.024691,0.230600,0.769400,-5.336419,-0.036389,-0.025296,-14.264350,-2.128057,15639.19054,-17641.89478,16741.833333,569.0,3.987698,523.0,5710.63935,327.524297,441.541109,1745.0,-4334.46065,-105.099259,208.221203,1136.0,-0.529418,249.495599,263.0,344.020090,420.570342,873.0,-104.328411,197.957617,1132.0,-10.160407,274.598057,260.0,310.838169,462.753846,872.0,-105.870991,218.496560
537,sim_2178,-1,4.0,0.004,3.0,True,7,

In [22]:
df = df_all[df_all.margin_closeout == True].copy()
df.reset_index(inplace=True)
df

,index,sim_name,init_signal,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
0,0,sim_1,1,1.5,0.001,1.5,True,1,-10.230730,-9230.73000,35475.0,-0.288393,16.752417,0.333531,0.666469,-0.288393,-0.002348,-0.002175,-0.013405,-10.086325,1016.16000,-9233.16000,33.333333,35476.0,1.000000,11832.0,16.93000,1.876650,22.190754,23643.0,-14.00000,-1.371875,14.030834,17695.0,-0.288110,16.812150,5873.0,1.888209,22.841989,11822.0,-1.369274,13.816613,17780.0,-0.288674,16.692970,5959.0,1.865258,21.548918,11821.0,-1.374477,14.245072
1,1,sim_2,1,1.5,0.001,1.5,True,2,-10.230730,-9230.73000,35475.0,-0.288393,16.752417,0.333531,0.666469,-0.288393,-0.002348,-0.002175,-0.013405,-10.086325,1016.16000,-9233.16000,33.333333,21183.0,1.674739,11832.0,16.93000,1.876650,22.190754,23643.0,-14.00000,-1.371875,14.030834,17695.0,-0.288110,16.812150,5873.0,1.888209,22.841989,11822.0,-1.369274,13.816613,17780.0,-0.288674,16.692970,5959.0,1.865258,21.548918,11821.0,-1.374477,14.245072
2,2,sim_3,-1,1.5,0.001,1.5,True,3,-12.319860,-11319.86000,35474.0,-0.347293,16.752889,0.333540,0.666460,-0.347293,0.006884,0.009342,0.007457,-12.065229,1023.16000,-11321.50000,66.666667,16746.0,2.118416,11832.0,22.58000,2.279707,22.189571,23642.0,-24.54000,-1.662015,14.032019,17694.0,-0.347382,16.812592,5873.0,2.288667,22.839775,11821.0,-1.657043,13.818120,17780.0,-0.347204,16.693476,5959.0,2.270876,21.548750,11821.0,-1.666986,14.245918
3,3,sim_4,-1,1.5,0.002,1.5,True,1,-3.517410,-2517.41000,9466.0,-0.371584,62.779527,0.350518,0.649482,-0.371584,-0.009140,-0.006092,-0.472808,-3.486658,1014.47000,-2522.64000,33.333333,9467.0,1.000000,3318.0,17.89000,3.463162,80.180229,6148.0,-11.95000,-2.441148,53.388582,4714.0,-0.369650,61.044972,1640.0,3.512341,78.903659,3074.0,-2.440719,51.517241,4752.0,-0.373502,64.500210,1678.0,3.415095,81.427890,3074.0,-2.441578,55.259922
4,4,sim_5,-1,1.5,0.002,1.5,True,2,-3.517410,-2517.41000,9466.0,-0.371584,62.779527,0.350518,0.649482,-0.371584,-0.009140,-0.006092,-0.472808,-3.486658,1014.47000,-2522.64000,33.333333,5734.0,1.651029,3318.0,17.89000,3.463162,80.180229,6148.0,-11.95000,-2.441148,53.388582,4714.0,-0.369650,61.044972,1640.0,3.512341,78.903659,3074.0,-2.440719,51.517241,4752.0,-0.373502,64.500210,1678.0,3.415095,81.427890,3074.0,-2.441578,55.259922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,535,sim_2176,-1,4.0,0.003,3.0,True,8,1.318534,2318.53445,2273.0,0.580086,261.448306,0.229212,0.770788,0.580086,-0.012775,-0.010266,-3.588886,-2.463724,6931.59064,-3525.18327,7175.066667,583.0,3.900515,521.0,2055.65660,211.613791,441.865643,1752.0,-1095.63268,-62.175942,207.796804,1138.0,-0.258045,248.965729,262.0,206.664346,421.377863,876.0,-62.145792,197.399543,1135.0,1.420431,273.963877,259.0,216.620565,462.590734,876.0,-62.206093,218.194064
536,536,sim_2177,1,4.0,0.003,3.0,True,9,-12.102999,-11102.99902,2268.0,-5.336419,262.024691,0.230600,0.769400,-5.336419,-0.036389,-0.025296,-14.264350,-2.128057,15639.19054,-17641.89478,16741.833333,569.0,3.987698,523.0,5710.63935,327.524297,441.541109,1745.0,-4334.46065,-105.099259,208.221203,1136.0,-0.529418,249.495599,263.0,344.020090,420.570342,873.0,-104.328411,197.957617,1132.0,-10.160407,274.598057,260.0,310.838169,462.753846,872.0,-105.870991,218.496560
537,537,s

In [23]:
df0 = df_all[df_all.margin_closeout == False].copy()
df0.reset_index(inplace=True)
df0

,index,sim_name,init_signal,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration


In [24]:
# figs1 = [
#             [px.scatter(df, x="cushion", y="return_%", color="init_signal", symbol="streak_limit"),
#             px.scatter(df, x="cushion", y="expectancy", color="init_signal", symbol="streak_limit")],
#             [px.scatter(df, x="cushion", y="win_%", color="init_signal", symbol="streak_limit"),
#             px.scatter(df, x="cushion", y="loss_%", color="init_signal", symbol="streak_limit")]
#     ]

# fig = make_subplots(
#     rows=2, cols=2,
#     subplot_titles=("return_%", "expectancy", "win_%", "loss_%"))

# for i, f1 in enumerate(figs1):
#     for j, f2 in enumerate(f1):
#         for trace in range(len(f2["data"])):
#             fig.append_trace(f2["data"][trace], row=i+1, col=j+1)
        
# fig.show()

In [25]:
px.scatter(df, x = 'rr', y = 'win_%', color = 'risk')

In [26]:
# win_% decreases with rr, lower rr higher win_%

In [27]:
px.scatter(df, x = 'risk', y = 'win_%', color='rr')

In [28]:
# win_% increases with risk if rr <= 2, higher risk higher win_% if rr <= 2

In [29]:
px.scatter(df, x = 'cushion', y = 'expectancy', color = 'rr', facet_col='risk')

In [30]:
px.scatter(df, x = 'cushion', y = 'expectancy', color = 'risk', facet_col='streak_limit')

In [31]:
# higher cushion, higher expectancy if risk >= 0.002

In [32]:
px.scatter(df, x = 'risk', y = 'expectancy', color = 'streak_limit', facet_col='streak_limit')

In [33]:
px.scatter(df, x = 'cushion', y = 'expectancy', color = 'streak_limit', facet_col='risk')

In [34]:
px.scatter(df, x = 'risk', y = 'max_loss')

In [35]:
# max loss decreases with risk, higher risk lower max loss

In [36]:
px.scatter(df, x='rr', y='max_loss', facet_col='streak_limit')

In [37]:
# max loss increases with lower rr but only for higher streak limits

In [38]:
px.scatter(df, x = 'risk', y = 'max_loss', color = 'streak_limit', facet_col='rr')

In [39]:
# max_loss decreases with risk, higher risk lower max loss

In [40]:
def plot_plots(df):  
    px.scatter(df, x = 'streak_limit', y = 'expectancy', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'return_%', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_margin_used', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'min_ac_bal', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'final_ac_bal', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_loss', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_win', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'sharpe_ratio', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'sortino_ratio', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'calmar_ratio', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_drawdown', color = 'risk', facet_col='rr').show()

In [41]:
df1 = df[(df.risk>=0.003) & (df.rr <= 2) & (df.cushion==2)]
df1

,index,sim_name,init_signal,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
42,42,sim_43,-1,2.0,0.003,1.5,True,1,-1.429460,-429.46000,4215.0,-0.339136,140.912930,0.373191,0.626809,-0.339136,0.015257,0.128970,2.377986e+01,-1.490188,1000.00000,-488.61000,33.333333,4216.0,1.000000,1573.0,15.01000,4.981113,177.153210,2642.0,-17.18000,-3.506718,119.336109,2106.0,-0.330014,136.801045,785.0,5.010051,169.862420,1321.0,-3.503331,117.154428,2109.0,-0.348246,145.018966,788.0,4.952284,184.416244,1321.0,-3.510106,121.517790
43,43,sim_44,1,2.0,0.003,1.5,True,2,-1.520876,-520.87614,4213.0,-0.360996,140.979824,0.373606,0.626394,-0.360996,0.003140,0.003502,1.145145e-01,-1.602950,1008.51164,-608.08214,44.466667,2575.0,1.636505,1574.0,20.02334,5.658632,177.080686,2639.0,-22.91812,-3.951331,119.447897,2106.0,-0.341542,136.814340,786.0,5.689416,169.726463,1320.0,-3.932704,117.216667,2107.0,-0.380440,145.143332,788.0,5.627926,184.416244,1319.0,-3.969972,121.680819
44,44,sim_45,1,2.0,0.003,1.5,True,3,-1.813799,-813.79850,4213.0,-0.430524,140.979824,0.373606,0.626394,-0.430524,0.029996,0.089088,8.948113e-01,-1.920377,1037.65504,-955.03390,103.733333,2066.0,2.039690,1574.0,40.20704,7.664935,177.080686,2639.0,-31.40008,-5.258964,119.447897,2106.0,-0.391701,136.814340,786.0,7.809591,169.726463,1320.0,-5.275197,117.216667,2107.0,-0.469329,145.143332,788.0,7.520647,184.416244,1319.0,-5.242719,121.680819
45,45,sim_46,-1,2.0,0.004,1.5,True,1,-1.028360,-28.36000,2404.0,-0.427770,247.151830,0.368552,0.631448,-0.427770,0.020359,4.410638,7.588353e+01,-1.138532,1005.83000,-139.34000,33.333333,2405.0,1.000000,886.0,15.01000,6.558646,297.371332,1518.0,-19.64000,-4.505481,217.840580,1201.0,-0.416311,236.322231,442.0,6.587489,285.454751,759.0,-4.494941,207.710145,1203.0,-0.439210,257.963425,444.0,6.529932,309.234234,759.0,-4.516021,227.971014
46,46,sim_47,-1,2.0,0.004,1.5,True,2,-1.094021,-94.02106,2404.0,-0.455084,247.151830,0.368552,0.631448,-0.455084,-0.022962,-0.021428,-4.113376e+00,-1.231272,1009.76786,-233.53134,44.466667,1463.0,1.643882,886.0,20.02334,7.466504,297.371332,1518.0,-26.19976,-5.078619,217.840580,1201.0,-0.424026,236.322231,442.0,7.500903,285.454751,759.0,-5.039070,207.710145,1203.0,-0.486090,257.963425,444.0,7.432259,309.234234,759.0,-5.118167,227.971014
47,47,sim_48,-1,2.0,0.004,1.5,True,3,-1.361401,-361.40134,2404.0,-0.566307,247.151830,0.368552,0.631448,-0.566307,0.006798,0.008904,1.189292e+00,-1.558336,1033.29028,-576.92344,103.733333,1176.0,2.045068,886.0,36.41040,10.113138,297.371332,1518.0,-32.64488,-6.799500,217.840580,1201.0,-0.451293,236.322231,442.0,10.397729,285.454751,759.0,-6.769170,207.710145,1203.0,-0.681129,257.963425,444.0,9.829829,309.234234,759.0,-6.829831,227.971014
54,54,sim_55,1,2.0,0.003,2.0,True,1,-1.618670,-618.67000,3266.0,-0.495612,181.857930,0.300980,0.699020,-0.495612,-0.017498,-0.012768,-1.461604e+10,-1.654483,1026.94000,-672.12000,33.333333,3267.0,1.000000,983.0,15.01000,6.525056,256.579858,2283.0,-17.18000,-3.518528,149.684625,1631.0,-0.486542,177.388719,489.0,6.579571,252.269939,1142.0,-3.512224,145.324869,1635.0,-0.504661,186.316208,494.0,6.471093,260.846154,1141.0,-3.524838,154.048203
55,55,sim_56,-1,2.0,0.003,2.0,True,2,-1.647610,-647.61000,3267.0,-0.504319,181.802265,0.300275,0.699725,-0.504319,0.032228,0.065873,7.621853e-01,-1.702465,1000.00000,-701.06000,33.333333,1905.0,1.715486,981.0,15.01000,6.524169,256.840979,22

In [42]:
plot_plots(df1)

In [43]:
df2 = df[(df.risk>=0.003) & (df.rr > 2)]# & (df.cushion==2) & (df.streak_limit<=7)]
df2.head()

,index,sim_name,init_signal,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
30,30,sim_31,-1,1.5,0.003,3.0,True,1,-1.181380,-181.3800,2273.0,-0.519745,261.448306,0.229212,0.770788,-0.519745,-0.018187,-0.016572,-0.549747,-1.193212,1004.7500,-194.130,33.333333,2274.0,1.000000,521.0,15.01,9.494280,441.865643,1752.0,-14.47,-3.497660,207.796804,1138.0,-0.507258,248.965729,262.0,9.520878,421.377863,876.0,-3.506541,197.399543,1135.0,-0.532264,273.963877,259.0,9.467375,462.590734,876.0,-3.488779,218.194064
31,31,sim_32,-1,1.5,0.003,3.0,True,2,-0.999220,0.7800,2273.0,-0.439604,261.448306,0.229212,0.770788,-0.439604,0.021573,0.098279,4.152892,-1.010820,1000.0000,-10.785,33.333333,1279.0,1.777952,521.0,14.09,7.331094,441.865643,1752.0,-11.55,-2.750411,207.796804,1138.0,-0.456968,248.965729,262.0,7.352786,421.377863,876.0,-2.792763,197.399543,1135.0,-0.422194,273.963877,259.0,7.309151,462.590734,876.0,-2.708059,218.194064
32,32,sim_33,1,1.5,0.003,3.0,True,3,-0.947423,52.5775,2268.0,-0.417735,262.024691,0.230600,0.769400,-0.417735,-0.060537,-0.131376,-0.366654,-0.958860,1014.0925,41.720,33.333333,950.0,2.388421,523.0,15.01,7.263193,441.541109,1745.0,-14.47,-2.719812,208.221203,1136.0,-0.414960,249.495599,263.0,7.251901,420.570342,873.0,-2.724679,197.957617,1132.0,-0.420519,274.598057,260.0,7.274615,462.753846,872.0,-2.714940,218.496560
33,33,sim_34,-1,1.5,0.004,3.0,True,1,-0.739210,260.7900,1273.0,-0.580683,466.549882,0.230165,0.769835,-0.580683,-0.077828,-0.310248,-0.395048,-0.769422,1000.0000,230.530,33.333333,1274.0,1.000000,293.0,22.18,12.556519,767.737201,980.0,-19.64,-4.508439,376.501020,636.0,-0.557673,430.941824,146.0,12.625959,720.116438,490.0,-4.485857,344.779592,637.0,-0.603658,502.102041,147.0,12.487551,815.034014,490.0,-4.531020,408.222449
34,34,sim_35,-1,1.5,0.004,3.0,True,2,-0.692470,307.5300,1273.0,-0.543967,466.549882,0.230165,0.769835,-0.543967,-0.086760,-0.312725,-0.382808,-0.711272,1000.0000,287.570,33.333333,712.0,1.789326,293.0,22.18,9.503515,767.737201,980.0,-19.64,-3.547959,376.501020,636.0,-0.514049,430.941824,146.0,9.532021,720.116438,490.0,-3.507367,344.779592,637.0,-0.573838,502.102041,147.0,9.475204,815.034014,490.0,-3.588551,408.222449


In [44]:
plot_plots(df2)

In [45]:
df3 = df[(df.risk>=0.003) & (df.rr == 1.5) & (df.cushion==2) & (df.streak_limit==5)]
df3

,index,sim_name,init_signal,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
223,223,sim_1044,1,2.0,0.003,1.5,True,5,-4.812785,-3812.78494,4213.0,-1.142365,140.979824,0.373606,0.626394,-1.142365,0.010878,0.030015,0.498585,-4.464979,1207.56470,-4184.18690,564.8,1742.0,2.419059,1574.0,121.99680,14.794647,177.080686,2639.0,-139.95744,-10.647805,119.447897,2106.0,-1.141927,136.814340,786.0,14.662247,169.726463,1320.0,-10.552594,117.216667,2107.0,-1.142804,145.143332,788.0,14.926710,184.416244,1319.0,-10.743089,121.680819
226,226,sim_1047,1,2.0,0.004,1.5,True,5,-2.694493,-1694.49250,2403.0,-1.121304,247.254682,0.368706,0.631294,-1.121304,-0.007755,-0.006360,-0.687851,-2.788765,1186.88504,-2123.05806,564.8,982.0,2.448065,886.0,155.20704,20.292780,297.478555,1517.0,-139.95744,-13.628145,217.921556,1201.0,-0.622576,236.330558,442.0,21.204531,285.669683,759.0,-13.333486,207.598155,1202.0,-1.619616,258.169717,444.0,19.385136,309.234234,758.0,-13.923191,228.258575


In [46]:
def plot_plots(df):  
    px.scatter(df, x = 'init_signal', y = 'expectancy', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'return_%', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'max_margin_used', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'min_ac_bal', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'final_ac_bal', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'max_loss', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'max_win', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'sharpe_ratio', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'sortino_ratio', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'calmar_ratio', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'max_drawdown', color = 'risk', facet_col='rr').show()

In [47]:
plot_plots(df3)

win_% decreases with rr, lower rr higher win_%

win_% increases with risk, higher risk higher win_%

expectancy increases with risk, higher risk higher expectancy

init_signal has not effect on win_%


In [48]:
df1.sort_values(by='final_ac_bal')

,index,sim_name,init_signal,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
402,402,sim_2043,1,2.0,0.003,1.5,True,7,-12.173694,-11173.69402,4213.0,-2.889555,140.979824,0.373606,0.626394,-2.889555,0.027276,0.202804,1.032532e+00,-8.736482,1490.81693,-11533.67837,3075.033333,1641.0,2.567946,1574.0,736.16298,31.486542,177.080686,2639.0,-594.09644,-23.392766,119.447897,2106.0,-4.315288,136.814340,786.0,29.569587,169.726463,1320.0,-24.492190,117.216667,2107.0,-1.464499,145.143332,788.0,33.398631,184.416244,1319.0,-22.292509,121.680819
407,407,sim_2048,-1,2.0,0.004,1.5,True,9,-11.228662,-10228.66249,2404.0,-4.670825,247.151830,0.368552,0.631448,-4.670825,0.048246,0.112199,1.015940e+00,-1.580561,18375.13672,-10667.89605,16741.833333,900.0,2.672222,886.0,3490.67225,85.995377,297.371332,1518.0,-2476.11715,-57.589306,217.840580,1201.0,-2.821083,236.322231,442.0,95.868096,285.454751,759.0,-60.292251,207.710145,1203.0,-6.517491,257.963425,444.0,76.167130,309.234234,759.0,-54.886361,227.971014
416,416,sim_2057,1,2.0,0.003,2.0,True,9,-9.630290,-8630.29000,3266.0,-2.948650,181.857930,0.300980,0.699020,-2.948650,-0.014029,-0.008309,-3.619939e-01,-6.766000,1752.52000,-10105.03000,4266.666667,1023.0,3.193548,983.0,956.16000,52.659644,256.579858,2283.0,-961.28000,-26.892124,149.684625,1631.0,-4.624838,177.388719,489.0,50.147464,252.269939,1142.0,-28.078126,145.324869,1635.0,-1.276563,186.316208,494.0,55.146397,260.846154,1141.0,-25.705083,154.048203
415,415,sim_2056,-1,2.0,0.003,2.0,True,8,-7.688440,-6688.44000,3267.0,-2.353364,181.802265,0.300275,0.699725,-2.353364,0.015484,0.012080,6.205736e-02,-8.596482,1074.60000,-8163.18000,2133.333333,1040.0,3.142308,981.0,500.48000,37.876442,256.840979,2286.0,-511.36000,-19.617336,149.600612,1631.0,-2.562563,177.458614,488.0,37.712684,252.557377,1143.0,-19.757944,145.395451,1636.0,-2.144804,186.132641,493.0,38.038540,261.081136,1143.0,-19.476728,153.805774
224,224,sim_1045,-1,2.0,0.003,1.5,True,6,-7.467704,-6467.70432,4215.0,-1.771697,140.912930,0.373191,0.626809,-1.771697,-0.021228,-0.013865,-3.048390e-01,-7.842724,1019.01568,-6972.84346,1317.866667,1676.0,2.515513,1573.0,371.63840,21.692400,177.153210,2642.0,-289.40352,-15.741805,119.336109,2106.0,-1.222226,136.801045,785.0,22.809501,169.862420,1321.0,-15.503002,117.154428,2109.0,-2.320387,145.018966,788.0,20.579552,184.416244,1321.0,-15.980608,121.517790
405,405,sim_2046,1,2.0,0.004,1.5,True,7,-6.874892,-5874.89176,2403.0,-2.860962,247.254682,0.368706,0.631294,-2.860962,0.019386,0.296183,2.198619e+01,-6.642755,1133.68690,-6397.11732,3075.033333,923.0,2.604550,886.0,1051.66140,40.791883,297.478555,1517.0,-554.42851,-28.356295,217.921556,1201.0,-2.019463,236.330558,442.0,44.703918,285.669683,759.0,-29.228599,207.598155,1202.0,-3.701761,258.169717,444.0,36.897469,309.234234,758.0,-27.482841,228.258575
414,414,sim_2055,-1,2.0,0.003,2.0,True,7,-6.762830,-5762.83000,3267.0,-2.070043,181.802265,0.300275,0.699725,-2.070043,0.017550,0.276319,3.765152e+00,-6.010202,1239.97000,-6212.50000,1066.666667,1072.0,3.048507,981.0,323.84000,26.309898,256.840979,2286.0,-204.16000,-14.248836,149.600612,1631.0,-1.901637,177.458614,488.0,27.496783,252.557377,1143.0,-14.453193,145.395451,1636.0,-2.237934,186.132641,493.0,25.135051,261.081136,1143.0,-14.044479,153.805774
419,419,sim_2060,1,2.0,0.004,2.0,True,9,-5.060290,-4060.29000,1840.0,-2.750158,322.909239,0.317935,0.682065,-2.750158,0

In [49]:
df.calmar_ratio.max()

840064165.1160791

In [50]:
with open(PATH + 'sim_2250.pkl', 'rb') as f:
    tempdf = pkl.load(f)
tempdf = tempdf['results'].copy()

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Trading/forex_bot/outputs/sim_2250.pkl'

In [ ]:
trades = tempdf[tempdf.signal!=0].copy()
trades.head()

In [ ]:
trades.ac_bal.pct_change().shift(-1)
px.histogram(x=trades.ac_bal.pct_change().shift(-1))

In [ ]:
returns = trades['ac_bal'].shift(-1) - trades['ac_bal']
px.histogram(x=returns)

In [ ]:
df.sort_values(by='final_ac_bal')